In [1]:
import random
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
test = pd.read_csv("/content/gdrive/MyDrive/Humana/original_dataset/2021_Competition_Holdout.csv",index_col=0)
test.head()
test_copy=test.copy()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,8,9,11,13,16,20,26,28,29,31,33,51,55,58,62,64,66,75,82,85,102,124,131,132,135,159,173,179,191,208,209,210,219,233,239,246,254,260,284,287,292,296,304,305,307,308,322,333,343,344,349,351,352,354) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
test.head(10)

,ID,auth_3mth_post_acute_dia,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ip_snf_net_paid_pmpm_cost_9to12m_b4,auth_3mth_acute_ckd,bh_ncal_pmpm_ct,src_div_id,total_bh_copay_pmpm_cost_t_9-6-3m_b4,bh_ip_snf_net_paid_pmpm_cost_3to6m_b4,cons_chmi,mcc_ano_pmpm_ct_t_9-6-3m_b4,auth_3mth_post_acute_trm,rx_maint_pmpm_cost_t_12-9-6m_b4,auth_3mth_post_acute_rsk,cons_ltmedicr,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,credit_bal_nonmtgcredit_60dpd,rx_bh_mbr_resp_pmpm_cost_9to12m_b4,rx_nonbh_pmpm_cost_t_9-6-3m_b4,atlas_pct_laccess_nhna15,auth_3mth_acute_vco,credit_hh_nonmtgcredit_60dpd,rx_bh_pmpm_ct_0to3m_b4,auth_3mth_dc_ltac,cons_lwcm10,auth_3mth_post_acute_inj,atlas_fsrpth14,auth_3mth_dc_home,atlas_wicspth12,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,cons_hxmioc,rx_generic_pmpm_cost_t_6-3-0m_b4,cmsd2_sns_digest_abdomen_pmpm_ct,atlas_ghveg_farms12,credit_hh_bankcardcredit_60dpd,...,bh_ip_snf_admit_days_pmpm_t_9-6-3m_b4,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4,zip_cd,auth_3mth_post_acute_ckd,atlas_pct_laccess_nhpi15,auth_3mth_post_acute_ner,auth_3mth_post_er,credit_num_consumerfinance_new,rx_gpi2_49_pmpm_cost_0to3m_b4,cons_chva,atlas_avghhsize,rx_overall_net_paid_pmpm_cost_6to9m_b4,atlas_ownhomepct,atlas_orchard_farms12,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4,atlas_pct_fmrkt_wic16,rx_gpi2_33_pmpm_ct_0to3m_b4,auth_3mth_post_acute_chf,rwjf_social_associate_rate,atlas_freshveg_farms12,auth_3mth_acute_ccs_042,auth_3mth_post_acute_inf,auth_3mth_acute_sns,days_since_last_clm_0to3m_b4,auth_3mth_dc_other,auth_3mth_bh_acute_mean_los,mcc_end_pct,auth_3mth_post_acute_gus,cons_lwcm07,atlas_pct_fmrkt_otherfood16,auth_3mth_post_acute_end,auth_3mth_acute_mus,atlas_perpov_1980_0711,atlas_pct_laccess_white15,auth_3mth_post_acute_mean_los,rx_gpi2_66_pmpm_ct,auth_3mth_acute_gus,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4,atlas_low_education_2015_update,race_cd
0,5M9OS1T3dLc09Ybc7013AI9f,0,0.00,9.735346,0.131562,100.000000,NaN,0.0,0,0.0,000,No Activity,0,NaN,No Activity,0,Dec_2x-4x,0,NaN,0.0,19.176009,1572.427508,0.0,Inc_1x-2x,0.462727,0,23.902786,1.0,0,NaN,0,0.768907,0,0.229941,Resolved,NaN,Inc_1x-2x,0.0,6.0,13.284579,...,No Activity,No_Change,37301,0,0.007196,0,0,0.343095,2.383333,NaN,2.43,1907.250000,68.314377,15.0,Inc_1x-2x,0.0,1.0,0,12.163237,7.0,0,0,0,11,0,0,0.0,0,NaN,100.000000,0,0,0.0,12.404716,0.0,0.333333,0,Dec_1x-2x,0.0,1.0
1,0M8f45b4a4Of6aS3TL6Ya9AI,0,0.00,6.264524,0.075427,75.000000,23.833889,0.0,0,0.0,000,No Activity,0,NaN,No Activity,0,No Activity,0,NaN,0.0,17.286603,2130.109835,0.0,No Activity,0.158575,0,26.833861,0.0,0,NaN,0,0.671256,0,0.082403,No Activity,NaN,New,0.0,15.0,12.515323,...,No Activity,No Activity,97023,0,0.033085,0,0,0.112537,0.000000,NaN,2.58,0.000000,54.660192,22.0,No Activity,75.0,0.0,0,13.250028,12.0,0,0,0,120,0,0,0.0,0,NaN,25.000000,0,0,0.0,35.363608,0.0,0.000000,0,No Activity,0.0,1.0
2,12MOS912Tb204LYacA2fIc22,0,0.00,6.136065,0.106704,85.000000,26.085757,0.0,0,0.0,000,No Activity,0,49.0,No Activity,0,Inc_2x-4x,0,8.0,0.0,19.106041,1806.043293,0.0,Dec_2x-4x,0.056065,0,10.346433,0.0,0,0.20606,0,0.550016,0,0.150577,No Activity,9.0,Inc_1x-2x,0.0,3.0,10.230300,...,No Activity,No Activity,43645,0,0.023967,0,0,0.154732,3.166667,130.0,2.38,46.926668,68.774570,10.0,Inc_1x-2x,0.0,0.33333334,0,13.486984,29.0,0,0,0,30,0,0,0.6,0,0.21852,60.000000,0,0,0.0,37.241558,0.0,0.000000,0,No_Change,0.0,1.0
3,M6O69S24T36LYec4AI5d0b56,0,0.00,1.244630,0.000000,NaN,64.748171,0.0,0,0.0,000,No Activity,0,31.0,No Activity,0,Dec_2x-4x,0,8.0,0.0,18.397765,2245.687853,0.0,Dec_1x-2x,0.015978,0,15.664997,0.0,0,0.16953,0,0.913622,0,0.228276,No Activity,7.0,Dec_1x-2x,0.0,0.0,8.367782,...,No Activity,No Activity,74316,0,0.003000,0,0,0.101670,0.000000,51.0,2.45,20.696667,78.339429,3.0,No Activity,NaN,0.33333334,0,12.066459,4.0,0,0,0,120,0,0,0.0,0,0.27983,NaN,0,0,0.0,3.726617,0.0,0.000000,0,Inc_1x-2x,0.0,1.0
4,45947aM37Oc8S3dTLYA00Ica,0,0.00,8.009543,0.061276,87.500000,61.518384,0.0,0,0.0,000,No A

In [5]:
test = test.replace("*",np.nan)

In [6]:
new_type = pd.read_csv('/content/gdrive/MyDrive/Humana/updated_type.csv',index_col=0)

In [7]:
cat = new_type["0"][0:66]
num = new_type["1"]

In [8]:
#drop covid_vaccination
cat = cat.drop(index=[62],axis=0)

In [9]:
test.dtypes.value_counts()

float64    202
object     108
int64       56
dtype: int64

In [10]:
test[cat] = test[cat].astype("object")
test[num] = test[num].astype("float64")

In [11]:
test_type = pd.concat([test[cat],test[num]],axis=1)

In [12]:
test_type.head()

,total_bh_copay_pmpm_cost_t_9-6-3m_b4,rx_maint_pmpm_cost_t_12-9-6m_b4,rx_nonbh_pmpm_cost_t_9-6-3m_b4,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,rx_generic_pmpm_cost_t_6-3-0m_b4,bh_ncdm_ind,atlas_retirement_destination_2015_upda,rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4,rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4,rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4,atlas_hiamenity,rx_overall_gpi_pmpm_ct_t_6-3-0m_b4,mcc_chf_pmpm_ct_t_9-6-3m_b4,bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4,rx_maint_pmpm_cost_t_6-3-0m_b4,cons_mobplus,rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4,rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4,atlas_hipov_1115,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4,total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4,atlas_type_2015_mining_no,rx_branded_pmpm_ct_t_6-3-0m_b4,med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4,atlas_low_employment_2015_update,total_allowed_pmpm_cost_t_9-6-3m_b4,cms_orig_reas_entitle_cd,oontwk_mbr_resp_pmpm_cost_t_6-3-0m_b4,hum_region,rx_nonmail_dist_gpi6_pmpm_ct_t_9-6-3m_b4,rej_med_er_net_paid_pmpm_cost_t_9-6-3m_b4,med_outpatient_mbr_resp_pmpm_cost_t_9-6-3m_b4,rx_nonbh_net_paid_pmpm_cost_t_6-3-0m_b4,atlas_type_2015_recreation_no,rx_gpi2_39_pmpm_cost_t_6-3-0m_b4,atlas_type_2015_update,total_ip_maternity_net_paid_pmpm_cost_t_12-9-6m_b4,atlas_population_loss_2015_update,rx_maint_pmpm_ct_t_6-3-0m_b4,...,cons_n2phi,bh_physician_office_copay_pmpm_cost_6to9m_b4,rwjf_income_inequ_ratio,rej_total_physician_office_visit_ct_pmpm_0to3m_b4,credit_num_nonmtgcredit_60dpd,credit_bal_autofinance_new,rwjf_men_hlth_prov_ratio,rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4,cmsd2_sns_genitourinary_pmpm_ct,bh_ncal_pct,atlas_pct_snap16,ccsp_227_pct,atlas_ghveg_sqftpth12,rx_days_since_last_script_6to9m_b4,atlas_orchard_acrespth12,atlas_pct_laccess_multir15,cons_cgqs,ccsp_065_pmpm_ct,atlas_medhhinc,rwjf_mental_distress_pct,atlas_pct_laccess_nhpi15,credit_num_consumerfinance_new,rx_gpi2_49_pmpm_cost_0to3m_b4,cons_chva,atlas_avghhsize,rx_overall_net_paid_pmpm_cost_6to9m_b4,atlas_ownhomepct,atlas_orchard_farms12,atlas_pct_fmrkt_wic16,rx_gpi2_33_pmpm_ct_0to3m_b4,rwjf_social_associate_rate,atlas_freshveg_farms12,days_since_last_clm_0to3m_b4,mcc_end_pct,cons_lwcm07,atlas_pct_fmrkt_otherfood16,atlas_pct_laccess_white15,auth_3mth_post_acute_mean_los,rx_gpi2_66_pmpm_ct,cons_ltmedicr
0,No Activity,Dec_2x-4x,Inc_1x-2x,Resolved,Inc_1x-2x,0,1,No Activity,Inc_1x-2x,No Activity,1,Inc_1x-2x,No Activity,No Activity,Inc_1x-2x,NaN,Dec_2x-4x,No Activity,0,No Activity,Dec_1x-2x,Dec_1x-2x,0,Inc_1x-2x,No Activity,0,Dec_2x-4x,1,No Activity,MID-SOUTH,No_Change,No Activity,No Activity,Inc_1x-2x,0,No Activity,0,No Activity,0,Inc_1x-2x,...,NaN,0.0,4.214352,0.000000,0.541015,1104.604369,0.000310,0.0,0.0,0.0,16.304351,7.5,0.000000,185.0,0.426431,1.354704,NaN,0.0,42637.0,0.135912,0.007196,0.343095,2.383333,NaN,2.43,1907.250000,68.314377,15.0,0.0,1.000000,12.163237,7.0,11.0,0.0,NaN,100.0,12.404716,0.0,0.333333,NaN
1,No Activity,No Activity,No Activity,No Activity,New,0,0,No Activity,New,No Activity,1,New,No Activity,No Activity,No Activity,NaN,No Activity,No Activity,0,No Activity,No Activity,No Activity,0,No Activity,No Activity,0,No Activity,1,No Activity,INTERMOUNTAIN,No Activity,No Activity,No Activity,No Activity,0,No Activity,0,No Activity,0,No Activity,...,NaN,0.0,4.052580,0.000000,0.451749,971.214884,0.002964,0.0,0.0,0.0,17.722003,0.0,NaN,120.0,0.343430,2.364684,NaN,0.0,54440.0,0.125001,0.033085,0.112537,0.000000,NaN,2.58,0.000000,54.660192,22.0,75.0,0.000000,13.250028,12.0,120.0,0.0,NaN,25.0,35.363608,0.0,0.000000,NaN
2,No Activity,Inc_2x-4x,Dec_2x-4x,No Activity,Inc_1x-2x,0,0,Dec_1x-2x,Inc_1x-2x,Inc_1x-2x,0,Inc_1x-2x,No Activity,No Activity,Inc_1x-2x,S,Inc_2x-4x,No Activity,0,No Activity,New,New,0,No Activity,No Activity,0,New,1,Inc_1x-2x,EAST CENTRAL,Inc_1x-2x,No Activity,New,Inc_1x-2x,0,New,0,No Activity,1,Inc_1x-2x,...,0.0,0.0,4.772224,0.333333,0.418568,1755.363716,0.001954,0.0,0.0,0.0,13.791816,4.8,62.159944,234.0,0.194514,

In [13]:
#"20-35":1,"35-65":2,"65-80":3,"80-105":4
bin=[20,35,65,80,105]
binlabel=["1","2","3","4"]
test_type["age_cutoff"] = pd.cut(test_type["est_age"],bins=bin,labels=binlabel,include_lowest=True)

In [14]:
test_type.dtypes.value_counts()

float64     201
object       65
category      1
dtype: int64

In [15]:
test_type["race_cd"] = test_type["race_cd"].replace(np.nan,0)
test_type["race_cd"] = test_type["race_cd"].astype("int64")

In [16]:
def valuation_formula(x, y):
    return str(x)+str(y)

test_type["race_age"] = test_type.apply(lambda row: valuation_formula(row['race_cd'], row['age_cutoff']), axis=1)

In [17]:
test_type["race_age"].unique()

array(['12', '13', '14', '23', '02', '22', '03', '43', '34', '53', '42',
       '24', '11', '21', '52', '63', '33', '01', '62', '54', '32', '04',
       '44', '51', '64', '41', '31', '61'], dtype=object)

In [18]:
test_type["race_age"] = test_type["race_age"].astype('int64')

In [19]:
test_type["race_age"] = test_type["race_age"].astype('object')

In [20]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler

test_std_scaled = test_type.copy()

# apply standardization on numerical features
for i in num:
    
    # fit on training data column
    scale = StandardScaler().fit(test_type[[i]])
    
    # transform the training data column
    test_std_scaled[i] = scale.transform(test_type[[i]])

In [21]:
test_scale = test_std_scaled.copy()

In [22]:
test_scale = test_scale.drop("age_cutoff",axis = 1)

In [23]:
test_scale[cat] = test_scale[cat].astype("object")
test_scale[num] = test_scale[num].astype("float64")

In [24]:
test_scale.dtypes.value_counts()

float64    201
object      66
dtype: int64

In [25]:
# retrieve list of categories for trend variables
cats = list(test_scale["rx_maint_pmpm_cost_t_12-9-6m_b4"].unique())
print(cats)

['Dec_2x-4x', 'No Activity', 'Inc_2x-4x', 'Dec_1x-2x', 'Inc_1x-2x', 'Resolved', 'No_Change', 'Inc_4x-8x', 'Dec_4x-8x', 'New', 'Inc_over_8x', 'Dec_over_8x']


In [26]:
# Merge Increase and Decrease trend
inc = []
dec = []
for c in cats:
    if "Inc" in c:
        inc.append(c)
    if "Dec" in c:
        dec.append(c)
        
print(inc)
print(dec)

['Inc_2x-4x', 'Inc_1x-2x', 'Inc_4x-8x', 'Inc_over_8x']
['Dec_2x-4x', 'Dec_1x-2x', 'Dec_4x-8x', 'Dec_over_8x']


In [27]:
# Replace inc, dec with new values

test_scale.replace(inc,"Inc",inplace = True)
test_scale.replace(dec,"Dec",inplace = True)

In [28]:
# Merge MABH_SEG: MAPD Behavioral Segmentation 

healthy_will = ['H1','H2','H3','H4','H5']
healthy_rel = ['H6','H7','H8']
chronic_will = ['C1','C2','C3','C4','C5']
chronic_rel = ['C6','C7']

test_scale.replace(healthy_will,"H_W",inplace = True)
test_scale.replace(healthy_rel,"H_R",inplace = True)
test_scale.replace(chronic_will,"C_W",inplace = True)
test_scale.replace(chronic_rel,"C_R",inplace = True)

In [29]:
# Merge CONS_HHCOMP : Indicator of the household composition based on family position and presence of children

test_scale["cons_hhcomp"].replace("U",np.nan,inplace = True)

In [30]:
test_scale["cons_hhcomp"].unique()

array([nan, 'L', 'A', 'B', 'D', 'G', 'I', 'J', 'F', 'H', 'K', 'C', 'E'],
      dtype=object)

In [31]:
# Merge hum_region: Member geographic information

test_scale.replace("PACIFIC","PR/PACIFIC",inplace = True)
test_scale.replace("PR","PR/PACIFIC",inplace = True)

In [32]:
# Merge race_cd : Unknown and null

test_scale["race_cd"].replace(0.0,np.nan,inplace = True)

In [33]:
# For testing purpose:

test_scale["race_cd"].unique()
#test_before_encoding["race_cd"].value_counts()

array([ 1.,  2., nan,  4.,  3.,  5.,  6.])

In [34]:
# RUN THIS CELL IF RACE_CD TYPE INCORRECT
test_scale[cat] = test_scale[cat].astype("object")
test_scale[num] = test_scale[num].astype("float64")
test_scale.dtypes.value_counts()

float64    201
object      65
int64        1
dtype: int64

In [35]:
test_scale["race_cd"].dtypes

dtype('O')

In [36]:
test_scale['race_age'] = test_scale['race_age'].astype('object')

In [37]:
def onehotprep(df):
  # Create a label encoder object
  le = LabelEncoder()
  le_count = 0
  # Iterate through the columns
  for col in df:
      if df[col].dtype == 'object':
        # If 2 or fewer unique categories
          if len(list(df[col].unique())) <= 2:
            # Train on the training data
              le.fit(df[col])
              # Transform both training
              df[col] = le.transform(df[col])
              # Keep track of how many columns were label encoded
              le_count += 1
                
  return print('%d columns were label encoded.' % le_count)

In [38]:
onehotprep(test_scale)

5 columns were label encoded.


In [39]:
test_encoding = pd.get_dummies(test_scale)
print(test_encoding.shape)

test_encoding.head(20)

(525158, 552)


,bh_ncdm_ind,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,rej_med_er_net_paid_pmpm_cost_t_9-6-3m_b4,total_ip_maternity_net_paid_pmpm_cost_t_12-9-6m_b4,sex_cd,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ncal_pmpm_ct,cons_chmi,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,credit_bal_nonmtgcredit_60dpd,rx_bh_mbr_resp_pmpm_cost_9to12m_b4,atlas_pct_laccess_nhna15,credit_hh_nonmtgcredit_60dpd,rx_bh_pmpm_ct_0to3m_b4,cons_lwcm10,atlas_fsrpth14,atlas_wicspth12,cmsd2_sns_digest_abdomen_pmpm_ct,atlas_ghveg_farms12,credit_hh_bankcardcredit_60dpd,total_outpatient_allowed_pmpm_cost_6to9m_b4,cons_cwht,atlas_netmigrationrate1016,atlas_pct_laccess_snap15,rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4,atlas_naturalchangerate1016,ccsp_236_pct,atlas_pct_laccess_hisp15,auth_3mth_dc_no_ref,rx_overall_mbr_resp_pmpm_cost,rx_overall_gpi_pmpm_ct_0to3m_b4,atlas_pct_laccess_hhnv15,credit_bal_consumerfinance,rwjf_uninsured_pct,rx_mail_mbr_resp_pmpm_cost_0to3m_b4,...,cons_hhcomp_G,cons_hhcomp_H,cons_hhcomp_I,cons_hhcomp_J,cons_hhcomp_K,cons_hhcomp_L,race_cd_1.0,race_cd_2.0,race_cd_3.0,race_cd_4.0,race_cd_5.0,race_cd_6.0,race_age_1,race_age_2,race_age_3,race_age_4,race_age_11,race_age_12,race_age_13,race_age_14,race_age_21,race_age_22,race_age_23,race_age_24,race_age_31,race_age_32,race_age_33,race_age_34,race_age_41,race_age_42,race_age_43,race_age_44,race_age_51,race_age_52,race_age_53,race_age_54,race_age_61,race_age_62,race_age_63,race_age_64
0,0,0,0,0,0,-0.065120,1.763877,0.848883,1.161712,NaN,-0.057237,NaN,-0.069205,0.165982,-0.946812,-0.147737,0.361461,0.143485,0.991917,NaN,0.231224,0.871978,-0.206746,0.283937,0.522528,-0.128344,NaN,-1.568219,-0.253312,-0.177412,-1.457030,-0.122948,0.443762,-0.016213,-0.421049,3.680120,1.527586,0.258693,0.611409,-0.321938,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,-0.065120,0.477020,-0.319468,0.357785,-1.415063,-0.057237,NaN,-0.069205,-0.085230,-0.431530,-0.147737,-0.008502,0.467975,-0.436070,NaN,-0.111378,-0.882748,-0.206746,1.632114,0.329829,-0.156300,NaN,-0.103725,1.206740,-0.177412,0.496639,-0.122948,1.687380,-0.016213,-0.423371,-0.753501,0.602917,-0.816878,0.201238,-0.321938,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,-0.065120,0.429392,0.331523,0.679356,-1.274542,-0.057237,-0.601115,-0.069205,0.156679,-0.730958,-0.147737,-0.133193,-1.357298,-0.436070,-0.690109,-0.536739,-0.071928,-0.206746,-0.165455,-0.242572,-0.156300,-0.335777,-0.183614,-0.348789,-0.177412,-1.177394,-0.122948,-0.295401,-0.016213,-0.345144,0.324947,0.245936,-0.912522,-1.251376,-0.151506,...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,-0.065120,-1.384177,-1.889345,NaN,1.138062,-0.057237,-1.241296,-0.069205,0.062508,-0.324740,-0.147737,-0.181954,-0.768496,-0.436070,-0.973000,0.738946,0.852178,-0.206746,-0.614847,-0.709135,-0.156300,-1.124970,0.432715,-1.048610,0.339738,-0.104856,-0.122948,-0.620227,-0.016213,-0.189102,-0.513846,-0.586706,-0.980725,1.365648,-0.215695,...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,-0.065120,1.124011,-0.613990,0.759749,0.936517,-0.057237,-0.956771,-0.069205,0.548892,0.633888,-0.147737,0.390118,0.374181,-0.436070,0.763608,-0.360005,-0.832582,-0.206746,-0.465050,1.114489,-0.156300,-1.190736,0.668534,1.221419,0.445635,-0.436966,-0.122948,-0.157387,-0.016213,1.010683,0.444775,-0.854589,1.425307,1.226254,-0.189134,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,-0.065120,-0.656976,0.956890,-0.446141,0.204072,-0.057237,0.074631,-0.069205,-0.603235,-0.359391,-0.147737,-0.104488,-0.715698,-0.436070,1.068414,0.173575,-0.650567,-0.206746,1.032925,-0.462792,-0.156300,0.190352,-0.239852,-0.878116,-0.177412,-0.183546,-0.122948,-0.511143,-0.016213,-0.423371,-0.873329,-0.567357,-1.019630,-0.610205,-0.321938,...,0,0,0,0,

In [40]:
train=pd.read_csv('/content/gdrive/MyDrive/Humana/train_post_oversampling_race.csv',index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [41]:
train_cols = train.columns

In [42]:
test_cols = test_encoding.columns

In [43]:
common_col = list(set(train_cols).intersection(set(test_cols)))

In [44]:
len(common_col)

503

In [45]:
# to list cols in train but not in holdout
add_cols = []
for col in train_cols:
  if col not in common_col:
    add_cols.append(col)

len(add_cols)

1

In [46]:
add_cols

['covid_vaccination']

In [47]:
test_common = test_encoding[common_col]

In [48]:
test_clean = test_common

In [49]:
test_clean.to_csv('test_clean_oversampling_race.csv')
!cp test_clean_oversampling_race.csv "/content/gdrive/MyDrive/Humana"